In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a = 1
medium = td.Medium(permittivity=11.56)
run = False

In [ ]:
file = r"H:\phd stuff\structures\SHU_2D\chi_0.30_N_10000_posics.dat"
data = []
centers = []
slicing= 0.1

with open(file,"r") as f: 
    lines = f.read().splitlines() 
    i=0
    index = 1
    num_samples = int(lines[index].split()[0])

    while num_samples>0 and i < 10:
        try:
            num_samples = int(lines[index].split()[0])
            info = lines[index]
            old_index = index
            index += num_samples+1
            li = (lines[old_index+1:index])
            centers = np.array([list(map(float, item.split())) for item in li])
            data += [{
                "centers":centers,
                "Lx":float(lines[old_index].split()[1]),
                "Ly":float(lines[old_index].split()[2]),
                "points":float(lines[old_index].split()[0]),
            }]
            i+=1
        except:
            num_samples = 0

In [ ]:
lambdas =  a/np.array([0.285,0.44])

In [ ]:
sample = 0
project_name = r"08_27_2024 chi 0.3 N1000 Moving Beam Experiments"

runtime_ps = 35e-12
min_steps_per_lambda = 20

for k,item in enumerate(data):
 for center_beam in np.linspace(-40,40,16):
    run_name = f"Beam Moving {1/lambdas[0]:.3g} - {1/lambdas[1]:.3g} - Sample_{k} {runtime_ps*1e12}ps center - {center_beam}"

    if k !=sample:
        continue

    structure_1 = AM.loadAndRunStructure(key = tidy3dAPI
                ,direction="z", lambda_range=lambdas,
                box_size= data[k]['Lx']*a,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
               scaling=1,shuoff_condtion=1e-12, verbose=True,
               monitors=["flux"],
               freqs=50, 
               source="planewave", 
               width=0.25, ref_only=True
               )
           
    print(run_name)

    sim = structure_1.sim



    boundaries= td.BoundarySpec(
            x=td.Boundary(plus=td.Absorber(num_layers=600),minus=td.Absorber(num_layers=600)),
            y=td.Boundary(plus=td.Absorber(num_layers=600),minus=td.Absorber(num_layers=600)),
            z=td.Boundary.periodic(),
        )
    sim = sim.copy(update={'boundary_spec':boundaries})
    sim = sim.copy(update={'sources':[]})
    sim = sim.copy(update={'monitors':[]})


    Lx, Ly = data[k]['Lx']*slicing+40,data[k]['Ly']+4
    sim = sim.copy(update={'size':[Lx,Ly,0]})


    source = td.GaussianBeam(
            source_time = td.GaussianPulse(
                freq0=structure_1.freq0,
                fwidth=structure_1.freqw
            ),
            size= (0.,td.inf,td.inf),
            center=(-(Lx)*slicing/2 - lambdas[0],center_beam,0),
            direction='+',
            waist_radius=5,
            pol_angle=np.pi/2,
            name='planewave',
            )

    
    
    sim = sim.copy(update={'sources':[source]})

    time_monitorFieldOut = td.FieldTimeMonitor(
                center = (0,0,0),
                size = (Lx+40,Ly+40,0),
                    start=0,
                    stop=structure_1.t_stop,
                    interval=10,
                    fields=["Ex", "Ey", "Ez"],
                    name="time_monitorFieldOut",
                    
                )
    

    freq_monitorFieldOut = td.FieldMonitor(
                center = (0,0,0),
                size = (Lx,Ly,0),
                   freqs =structure_1.monitor_freqs,
                    fields=["Ex", "Ey", "Ez"],
                    name="freq_monitorFieldOut",
                    
                )
    
    sim = sim.copy(update={'monitors':[freq_monitorFieldOut]})
    
   
    
    cyl_group = []
    cyl_coords = []
    for x,y in data[k]['centers']:
        if x<=-data[k]['Ly']*(0.5-slicing):

            cyl_group.append(td.Cylinder(center=[x+data[k]['Ly']*(1-slicing)/2, y, 0], radius=0.189, length=td.inf))
            cyl_coords += [[x,y]]
    cylinders = td.Structure(geometry=td.GeometryGroup(geometries=cyl_group), medium=medium)

    break

    
    sim = sim.copy(update={'structures':[cylinders]})
    fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
    sim.plot(z=0, ax=ax)
    plt.show()


    if run:
        print("running ref...")
        id0 = web.upload(sim.copy(update={"structures":[]}), folder_name=project_name,task_name=run_name+'_0', verbose=True)
        web.start(task_id = id0)
        web.monitor(task_id=id0,verbose=True)

        id =web.upload(sim, folder_name=project_name,task_name=run_name, verbose=True)
        web.start(task_id = id)
        web.monitor(task_id=id,verbose=True)
    
        ids = (id0 if id0 else '') +'\n' + id
        incidence_folder = "z_incidence"
        file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}/{run_name}.txt"
        # Check if the folder exists
        if not os.path.exists( rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}"):
            os.makedirs(rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}")
            print(f"Folder '{project_name}/{incidence_folder}' created successfully.")

        # Open file in write mode
        with open(file_path, "w") as file:
            # Write the string to the file
            file.write(ids)





In [8]:
# data = {
#     "centers":cyl_coords
# }

In [9]:
# AM.create_hdf5_from_dict(data,"cyl_centers.h5")